# ReAct Agent

Adapted from https://github.com/opendatahub-io/llama-stack-demos/blob/main/demos/rag_agentic/notebooks/Level3_advance_agentic_with_Prompt_Chaining_ReAct.ipynb

In [1]:
import rich
import uuid
from llama_stack_client import Agent, LlamaStackClient, AgentEventLogger
from llama_stack_client.lib.agents.client_tool import client_tool
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.react.tool_parser import ReActOutput

In [2]:
# this may be missing from pyproject.toml, add it if you need it
#!uv pip install geocoder

In [3]:
@client_tool
def get_location(query: str = "location"):
    """
    Provide the location upon request.

    :param query: The query from user
    :returns: Information about user location
    """
    import geocoder
    
    try:
        g = geocoder.ip('me')
        if g.ok:
            return f"Your current location is: {g.city}, {g.state}, {g.country}" # can be modified to return latitude and longitude if needed
        else:
            return "Unable to determine your location"
    except Exception as e:
        return f"Error getting location: {str(e)}"

In [4]:
client = LlamaStackClient(
    base_url="http://localhost:8321",
)

In [5]:
models = client.models.list()
rich.print(models)

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/models "HTTP/1.1 200 OK"


[
    Model(
        identifier='llama3.2:3b-instruct-fp16',
        metadata={},
        api_model_type='llm',
        provider_id='ollama',
        type='model',
        provider_resource_id='llama3.2:3b-instruct-fp16',
        model_type='llm'
    ),
    Model(
        identifier='all-MiniLM-L6-v2',
        metadata={'embedding_dimension': 384.0},
        api_model_type='embedding',
        provider_id='ollama',
        type='model',
        provider_resource_id='all-minilm:latest',
        model_type='embedding'
    )
]

In [6]:
toolgroups = client.toolgroups.list()
rich.print(toolgroups)

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/toolgroups "HTTP/1.1 200 OK"


[
    ToolGroup(
        identifier='builtin::websearch',
        provider_id='tavily-search',
        type='tool_group',
        args=None,
        mcp_endpoint=None,
        provider_resource_id='builtin::websearch'
    ),
    ToolGroup(
        identifier='builtin::rag',
        provider_id='rag-runtime',
        type='tool_group',
        args=None,
        mcp_endpoint=None,
        provider_resource_id='builtin::rag'
    ),
    ToolGroup(
        identifier='builtin::wolfram_alpha',
        provider_id='wolfram-alpha',
        type='tool_group',
        args=None,
        mcp_endpoint=None,
        provider_resource_id='builtin::wolfram_alpha'
    ),
    ToolGroup(
        identifier='mcp::filesystem',
        provider_id='model-context-protocol',
        type='tool_group',
        args=None,
        mcp_endpoint=McpEndpoint(uri='http://localhost:8000/sse'),
        provider_resource_id='mcp::filesystem'
    )
]

In [12]:
tools = client.tools.list()
rich.print(tools)

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools "HTTP/1.1 500 Internal Server Error"
INFO:llama_stack_client._base_client:Retrying request to /v1/tools in 0.388742 seconds
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools "HTTP/1.1 500 Internal Server Error"
INFO:llama_stack_client._base_client:Retrying request to /v1/tools in 0.848228 seconds
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools "HTTP/1.1 500 Internal Server Error"


InternalServerError: Error code: 500 - {'detail': 'Internal server error: An unexpected error occurred.'}

## Baseline

In [13]:
client.toolgroups.register(
    toolgroup_id="builtin::websearch",
    provider_id="tavily-search",
)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/toolgroups "HTTP/1.1 200 OK"


In [14]:
toolgroups = client.toolgroups.list()
rich.print(toolgroups)

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/toolgroups "HTTP/1.1 200 OK"


[
    ToolGroup(
        identifier='builtin::websearch',
        provider_id='tavily-search',
        type='tool_group',
        args=None,
        mcp_endpoint=None,
        provider_resource_id='builtin::websearch'
    ),
    ToolGroup(
        identifier='builtin::rag',
        provider_id='rag-runtime',
        type='tool_group',
        args=None,
        mcp_endpoint=None,
        provider_resource_id='builtin::rag'
    ),
    ToolGroup(
        identifier='builtin::wolfram_alpha',
        provider_id='wolfram-alpha',
        type='tool_group',
        args=None,
        mcp_endpoint=None,
        provider_resource_id='builtin::wolfram_alpha'
    ),
    ToolGroup(
        identifier='mcp::filesystem',
        provider_id='model-context-protocol',
        type='tool_group',
        args=None,
        mcp_endpoint=McpEndpoint(uri='http://localhost:8000/sse'),
        provider_resource_id='mcp::filesystem'
    )
]

In [17]:
agent = Agent(
    client,
    model="llama3.2:3b-instruct-fp16",
    instructions="You are a helpful assistant.",
    tools=["builtin::websearch"],
)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools?toolgroup_id=builtin%3A%3Awebsearch "HTTP/1.1 200 OK"


In [18]:
response = agent.create_turn(
    messages=[{"role": "user", "content": "Are you able to search the web?"}],
    session_id=agent.create_session(f"tool-check-{uuid.uuid4()}"),
    stream=False,
)
rich.print(response)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/4e3c3c82-5cde-4c57-8e00-c562cde3a427/session "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/4e3c3c82-5cde-4c57-8e00-c562cde3a427/session/0cdc8d24-e47e-4553-a751-f13b77623cc3/turn "HTTP/1.1 200 OK"


Turn(
    input_messages=[UserMessage(content='Are you able to search the web?', role='user', context=None)],
    output_message=CompletionMessage(
        content='Yes, I can search the web for information on a wide range of topics. What would you like me to 
look up?',
        role='assistant',
        stop_reason='end_of_turn',
        tool_calls=[]
    ),
    session_id='0cdc8d24-e47e-4553-a751-f13b77623cc3',
    started_at=datetime.datetime(2025, 6, 6, 13, 44, 5, 331467, tzinfo=TzInfo(UTC)),
    steps=[
        InferenceStep(
            api_model_response=CompletionMessage(
                content='Yes, I can search the web for information on a wide range of topics. What would you like 
me to look up?',
                role='assistant',
                stop_reason='end_of_turn',
                tool_calls=[]
            ),
            step_id='ee48a994-2719-44b2-9be7-a2785a0a70b8',
            step_type='inference',
            turn_id='a7017ac2-f332-4612-af0f-270eeef900e2',
            completed_at=datetime.datetime(2025, 6, 6, 13, 44, 12, 986461, tzinfo=TzInfo(UTC)),
            started_at=datetime.datetime(2025, 6, 6, 13, 44, 5, 331797, tzinfo=TzInfo(UTC))
        )
    ],
    turn_id='a7017ac2-f332-4612-af0f-270eeef900e2',
    completed_at=datetime.datetime(2025, 6, 6, 13, 44, 13, 1077, tzinfo=TzInfo(UTC)),
    output_attachments=[]
)

In [20]:
agent = Agent(
    client, 
    model="llama3.2:3b-instruct-fp16",
    instructions="You are a helpful assistant. Use websearch tool to help answer questions.",
    tools=["builtin::websearch"],
)
user_prompts = [
    # "Are there any weather-related risks in my area?",
    "Are there any weather-related risks in my area that could disrupt network connectivity or system availability?",

]
session_id = agent.create_session(f"test-session-{uuid.uuid4()}")
for prompt in user_prompts:
    rich.print(f"User> {prompt}")

    response = agent.create_turn(
        messages=[{"role": "user", "content": prompt}],
        session_id=session_id,
        stream=True,
    )
    for log in AgentEventLogger().log(response):
        log.print()

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools?toolgroup_id=builtin%3A%3Awebsearch "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/45e4c7c3-ed03-4a14-a85c-83dafc08d25a/session "HTTP/1.1 200 OK"


User> Are there any weather-related risks in my area that could disrupt network connectivity or system 
availability?

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/45e4c7c3-ed03-4a14-a85c-83dafc08d25a/session/3f5eaa43-a542-47e4-9e70-dfb8d5bed956/turn "HTTP/1.1 200 OK"


inference> brave_search.call(query="weather-related risks near me")
tool_execution> Tool:brave_search Args:{'query': 'weather-related risks near me'}
tool_execution> Tool:brave_search Response:{"query": "weather-related risks near me", "top_k": [{"title": "Weather in near me", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Phumi Near Pisei', 'region': 'Koh Kong', 'country': 'Cambodia', 'lat': 11.1333, 'lon': 103.5833, 'tz_id': 'Asia/Phnom_Penh', 'localtime_epoch': 1749217299, 'localtime': '2025-06-06 20:41'}, 'current': {'last_updated_epoch': 1749216600, 'last_updated': '2025-06-06 20:30', 'temp_c': 26.5, 'temp_f': 79.6, 'is_day': 0, 'condition': {'text': 'Patchy rain nearby', 'icon': '//cdn.weatherapi.com/weather/64x64/night/176.png', 'code': 1063}, 'wind_mph': 4.0, 'wind_kph': 6.5, 'wind_degree': 265, 'wind_dir': 'W', 'pressure_mb': 1008.0, 'pressure_in': 29.78, 'precip_mm': 0.48, 'precip_in': 0.02, 'humidity': 90, 'cloud': 78, 'feelslike_c': 30.4, 'feelslik

## Prompt Chaining

In [21]:
agent = Agent(
    client,
    model="llama3.2:3b-instruct-fp16",
    instructions="""You are a helpful assistant. 
    When a user asks about their location, use the get_location tool. 
    When searching for nearby places, use the websearch tool.
    """,
    tools=[get_location, "builtin::websearch"],
)
user_prompts = [
    "Where am I?",
    "Are there any weather-related risks in my area?",
]
session_id = agent.create_session(f"prompt-chaining-session-{uuid.uuid4()}")
for prompt in user_prompts:
    rich.print(f"User> {prompt}")
    response = agent.create_turn(
        messages=[{"role": "user", "content": prompt}],
        session_id=session_id,
    )
    for log in EventLogger().log(response):
        log.print()

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools?toolgroup_id=builtin%3A%3Awebsearch "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/23c52dcc-743d-4c91-b15b-0ed5e47f1a64/session "HTTP/1.1 200 OK"


User> Where am I?

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/23c52dcc-743d-4c91-b15b-0ed5e47f1a64/session/b40d1dc6-cd6d-4a52-b8af-10e261013ef4/turn "HTTP/1.1 200 OK"


inference> [get_location(query="my current location")]


INFO:geocoder.base:Requested http://ipinfo.io/json
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/23c52dcc-743d-4c91-b15b-0ed5e47f1a64/session/b40d1dc6-cd6d-4a52-b8af-10e261013ef4/turn/b0d69d5b-5bae-4df1-8641-e5f935c6abdf/resume "HTTP/1.1 200 OK"


...

User> Are there any weather-related risks in my area?

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/23c52dcc-743d-4c91-b15b-0ed5e47f1a64/session/b40d1dc6-cd6d-4a52-b8af-10e261013ef4/turn "HTTP/1.1 200 OK"


inference> [get_location(query="weather risks near me")]


INFO:geocoder.base:Requested http://ipinfo.io/json
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/23c52dcc-743d-4c91-b15b-0ed5e47f1a64/session/b40d1dc6-cd6d-4a52-b8af-10e261013ef4/turn/bf9d38bc-2533-46bd-bba1-643e8fa22aea/resume "HTTP/1.1 200 OK"


tool_execution> Tool:get_location Args:{'query': 'weather risks near me'}
tool_execution> Tool:get_location Response:"Your current location is: Llinars del Vallès, Catalonia, ES"
inference> {"message": "There are no severe weather warnings for your location. However, there is a chance of scattered thunderstorms in the area today.", "probability": 0.7}<|python_tag|>{"message": "There are no severe weather warnings for your location. However, there is a chance of scattered thunderstorms in the area today.", "probability": 0.7}


### ReAct Agent 

This section demonstrates the ReAct (Reasoning and Acting) framework in action.

For example, when asked "Are there any weather-related risks in my area that could disrupt network connectivity or system availability?", the agent:
1. **Reasons** it needs location information first
2. **Acts** by calling the `get_location` client tool
3. **Observes** the location result
4. **Reasons** about the next step
5. **Acts** by calling the `websearch` tool with observed location
6. **Observes** and processes the search results

Unlike prompt chaining which follows fixed steps, ReAct dynamically breaks down tasks and adapts its approach based on the results of each step. This makes it more flexible and capable of handling complex, real-world queries effectively.

In [24]:
agent = ReActAgent(
    client=client,
    model="llama3.2:3b-instruct-fp16",
    instructions="You are a helpful assistant. Use the tools at your disposal to answer any questions.",
    tools=[get_location, "builtin::websearch"],
    response_format={
        "type": "json_schema",
        "json_schema": ReActOutput.model_json_schema(),
    },
)
user_prompts = [
    "Are there any weather-related risks in my area?",
]
session_id = agent.create_session(f"web-session-{uuid.uuid4()}")
for prompt in user_prompts:
    rich.print(f"User> {prompt}")
    response = agent.create_turn(
        messages=[{"role": "user", "content": prompt}],
        session_id=session_id,
    )
    for log in EventLogger().log(response):
        log.print()

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools?toolgroup_id=builtin%3A%3Awebsearch "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/03f704ac-cd02-4fb4-8bb5-b208e5299633/session "HTTP/1.1 200 OK"


User> Are there any weather-related risks in my area?

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/03f704ac-cd02-4fb4-8bb5-b208e5299633/session/b276713e-9524-49c0-b368-696315a0029c/turn "HTTP/1.1 200 OK"


inference> 500: Internal server error: An unexpected error occurred.
